In [9]:
import re

source_file = open('1.txt','r')
sent_tokens = source_file.readlines()

def pass1():

	macro_name_table, arg_list_arr, alas, macro_def_table = ([] for i in range(4))
	mdt_index, mnt_index, ala_index = (0 for i in range(3))
	intermediate_code = ""

	#print (sent_tokens)

	for sent in sent_tokens:

		mdt_entry = {'Index': 0, 'Definition': "null"}
		mnt_entry = {'Index': 0, 'Macro Name': "null", 'MDT Index': 0}
		ala_entry = {'Index': 0, 'Argument': "null", 'Value': '-'}
		sent = sent.strip()
		sent = sent.replace(", ", " ")
		if(';' in sent):
			sent = re.sub(';.*', '', sent) 						##will substitute blank whereever '; or . or *' is occured 

		word_tokens = re.findall(r"[\S]+", sent)  
		if("MACRO" in sent):				
			flag = 1
			continue

		if flag == 0:
			intermediate_code += sent+"\n"								##for intermedite code

		if flag >= 1:											##This will run when macro is encountered
			# sent = sent.replace("\t", " ")
			mdt_index += 1
			mdt_entry = {'Index': mdt_index, 'Definition': sent}
			macro_def_table.append(mdt_entry)					##appending the mdt entry into the list
			if flag == 1:
				macro_name = word_tokens[0]
				mnt_index += 1
				operands = word_tokens[1].split(',')
				mnt_entry = {'Index': mnt_index, 'Macro Name': macro_name, 'MDT Index': mdt_index}
				macro_name_table.append(mnt_entry)				##appending the mnt entry into the list
				for operand in operands:
					ala_index += 1
					ala_entry = {'Index': ala_index, 'Argument': operand, 'Value': '-'}
					alas.append(ala_entry)						
				arg_list_arr.append(alas)						##appending the ala into the list
			flag += 1

		if "MEND" in sent:
			flag = 0
			alas = []
			ala_index = 0
	print("\nIntermediate code after pass 1 :- \n")				
	print(intermediate_code)											##Printing the intermedite code

	#Output Code Pass 1
	print("\n\nMacro Name Table :- \n")
	print('{:10}'.format("Index"), '{:20}'.format("Macro Name"), '{:20}'.format("MDT Index"))
	for x in macro_name_table:
		print('{:10}'.format(str(x["Index"])), '{:20}'.format(x["Macro Name"]), '{:20}'.format(str(x["MDT Index"])))

	for alas in arg_list_arr:
		print("\n\nArgument List Array for "+macro_name_table[arg_list_arr.index(alas)]["Macro Name"]+":- \n")
		print('{:10}'.format("Index"), '{:20}'.format("Argument"), '{:20}'.format("Value"))
		for x in alas:
			print('{:10}'.format(str(x["Index"])), '{:20}'.format(x["Argument"]), '{:20}'.format(x["Value"]))

	print("\n\nMacro Definition Table :- \n")
	print('{:10}'.format("Index"), '{:20}'.format("Definiton"))
	for x in macro_def_table:
		print('{:10}'.format(str(x["Index"])), '{:20}'.format(x["Definition"]))

	

	pass2(intermediate_code, macro_name_table, macro_def_table, arg_list_arr)

def pass2(intermediate_code, macro_name_table, macro_def_table, arg_list_arr):
	final_code = ""
	sent_tokens = intermediate_code.split('\n')  		##spliting the intermediate code of pass1 into tokens 
	for sent in sent_tokens:
		# sent = sent.replace('\t', " ")
		word_tokens = re.findall(r"[\S]+", sent)
		flag = -1
		for word in word_tokens:
			mnt_entries = [mnt for mnt in macro_name_table if mnt["Macro Name"] == word] 
															##checks whether the word is in macro_name_table
			if len(mnt_entries)>0:
				mnt_entry = mnt_entries[0]
				#print(mnt_entry)		
				flag = 1
				continue
			if flag == 1:
				operands = word
				ops = operands.split(',')					##operands of the macro call		

				for ala_entry in arg_list_arr[mnt_entry["Index"]-1]:
					if ala_entry["Index"] <= len(ops):
						ala_entry["Value"] = ops[ala_entry["Index"]-1]
				for mdt_entry in macro_def_table:	
					if mdt_entry["Index"] == mnt_entry["MDT Index"]:	##checks if mdt index matches in macro_def_table 
						flag = 2
						continue
					if mdt_entry["Index"] >= mnt_entry["MDT Index"] and 'MEND' in mdt_entry["Definition"]:
						flag = 1
						break	
					if flag == 2:																		
						sent = mdt_entry["Definition"]						##defination of macro will go in sent
						for ala_entry in arg_list_arr[mnt_entry["Index"]-1]:
							if ala_entry["Argument"] in mdt_entry["Definition"]:
								sent = sent.replace(ala_entry["Argument"], ala_entry["Value"]) ##replace arguments with operands						
						final_code += sent +'\n'
				flag = 0
		if flag == -1:
			final_code += sent +'\n'

	#Output pass 2
	print("\n\nArgument List Arrays for Pass 2 :- ")
	for alas in arg_list_arr:
		print("\nArgument List Array for "+macro_name_table[arg_list_arr.index(alas)]["Macro Name"]+":- \n")
		print('{:10}'.format("Index"), '{:20}'.format("Argument"))
		for x in alas:
			print('{:10}'.format(str(x["Index"])), '{:20}'.format(x["Value"]))
	print("\nOutput Code after second pass :- \n")
	print(final_code)

	

pass1()

['MACRO   \n', 'INCR    &A,&B,&REG\n', 'MOVER   &REG,&A\n', 'ADDS1   &A,&B\n', 'MOVEM   &REG,&A\n', 'MEND\n', 'MACRO   \n', 'ADDS1   &C,&D\n', 'MOVER   AREG,&C\n', 'ADD AREG,&D\n', 'MOVEM   AREG,&D\n', 'WRITE   &D\n', 'MEND\n', 'MACRO   \n', 'SUBS1   &C,&D  \n', 'MOVER   BREG,&C\n', 'SUBS    BREG,&D\n', 'MOVEM   BREG,&D\n', 'WRITE   &D\n', 'MEND\n', 'START   200\n', 'READ    N1\n', 'READ    N2\n', 'ADDS1   N1,N2\n', 'SUBS1   N1,N2\n', 'READ    N3\n', 'READ    N4\n', 'INCR    N3,N4,DREG\n', 'STOP\n', 'N1  DS  2\n', 'N2  DS  2\n', 'N3  DS  2\n', 'N4  DS  2\n', 'END\n']

Intermediate code after pass 1 :- 

START   200
READ    N1
READ    N2
ADDS1   N1,N2
SUBS1   N1,N2
READ    N3
READ    N4
INCR    N3,N4,DREG
STOP
N1  DS  2
N2  DS  2
N3  DS  2
N4  DS  2
END



Macro Name Table :- 

Index      Macro Name           MDT Index           
1          INCR                 1                   
2          ADDS1                6                   
3          SUBS1                12                  


In [14]:
import re #regular expression library to detect re
import pandas as pd # pandas to convert dict to  
file = open('input.txt','r') 
symbol = dict()
var = list()
literal = dict()
var1 = list()
LocCount = 0 #locationn counter to keep track of address
label = dict()
regx_literal = re.compile(r'=[0-9]')   # re for literal
regx_symbol = re.compile(r'=[A-Z]')    # re for symbol
for line in file:
    line.strip()     # remove white space from left and right
    words = line.split()    # split string into word        
    if len(words)>3 :
        symbol[str(words[0])] = LocCount        
    if line.startswith('ORIGIN'):
            no = words[1].split('+')
            if no[0] in symbol.keys():
                LocCount = symbol[no[0]]+int(no[1])
            continue    
    if line.startswith('START'):            # if line starts with start initialize loc counter to corresponding value
        LocCount = int(words[-1])
        continue        
    if regx_symbol.search(line):            # if line contains symbol store it in symbol table
        var.append(words[-1])
        symbol[str(words[-1])] = 0
        
        
    if regx_literal.search(line):           # if line contains literal store it in symbol table
        var1.append(words[-1])
        literal[str(words[-1])] = 0        
    if 'DC' in line:        # if line contains DS or DC assign address to the symbols stored previously
        symbol[str(words[0])] = LocCount        
    if 'DS' in line:
        symbol[str(words[0])] = LocCount
        LocCount += int(words[-1])
        continue       
    if line.startswith('LTORG'):              # if line contains LTORG word assign address to all the literals stored previously
        for w in var1:
            literal[w] = LocCount
            LocCount += 1
        LocCount -= 1             
    if line.startswith('END'):              # if line contains end word assign address to all the literals stored previously
        for w in var1:
            if literal.get(w)==0:
                literal[w] = LocCount
                LocCount += 1       
    LocCount+=1        
file.close()
file = open('input.txt','r')
op = open('output.txt','w')

optable = {'START':"('AD',1)",
           'END':"('AD',2)",
          'LTORG':"('AD',5)",
          'ORIGIN':"('AD',3)",
          'EQU':"('AD',4)",
          'DC':"('DL', 01)",
          'DS':"('DL', 02)",
          'ADD':"('IS', 01)",
          'SUB':"('IS', 02)",
          'MOVER':"('IS', 04)",
          'MOVEM':"('IS', 05)",
          'READ':"('IS', 09)",
          'PRINT':"('IS', 10)"
          }

register = ['AREG','BREG']
regx_constant = re.compile('[0-9]+')
for line in file:
    line.strip()    
    words = line.split()
    ic = ""        
    if words[0] in optable.keys():
        if words[0]=='ORIGIN':
            ic+=optable[words[0]]+" "+words[1]
            op.write(ic+'\n'